In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as stats
import missingno as msno

warnings.filterwarnings('ignore')

In [2]:
def norm(x: np.ndarray) -> bool:
    return stats.shapiro(x)[1] > 0.05


def correlation_ratio(categorical_series, numerical_series):
    categories = categorical_series.unique()
    total_mean = numerical_series.mean()

    ss_between = sum(len(numerical_series[categorical_series == category]) *
                     (numerical_series[categorical_series == category].mean() - total_mean) ** 2
                     for category in categories)

    ss_total = sum((numerical_series - total_mean) ** 2)

    eta_squared = ss_between / ss_total if ss_total != 0 else 0

    return np.sqrt(eta_squared)


def corr_matrix(df: pd.DataFrame, numeric: list[str], nominal: list[str]):
    cols = numeric + nominal
    corr_df = pd.DataFrame(index=cols, columns=cols)

    for col1 in cols:
        for col2 in cols:
            if col1 == col2:
                corr_df.loc[col1, col2] = 1.0
            elif col1 in numeric and col2 in numeric:
                norm1 = norm(df[col1].dropna().values)
                norm2 = norm(df[col2].dropna().values)
                if norm1 and norm2:
                    corr_value, _ = stats.pearsonr(df[col1].dropna(), df[col2].dropna())
                else:
                    corr_value, _ = stats.spearmanr(df[col1].dropna(), df[col2].dropna())
                corr_df.loc[col1, col2] = corr_value
                corr_df.loc[col2, col1] = corr_value
            elif col1 in nominal and col2 in nominal:
                contingency_table = pd.crosstab(df[col1], df[col2])
                _, p, _, _ = stats.chi2_contingency(contingency_table)
                corr_df.loc[col1, col2] = p
                corr_df.loc[col2, col1] = p
            else:
                num_col, cat_col = (col1, col2) if col1 in numeric else (col2, col1)
                corr_value = correlation_ratio(df[cat_col], df[num_col])
                corr_df.loc[col1, col2] = corr_value
                corr_df.loc[col2, col1] = corr_value

    corr_df = corr_df.astype(float)


    plt.figure(figsize=(10, 8))
    cmap = sns.diverging_palette(220, 20, as_cmap=True)
    sns.heatmap(corr_df, annot=True, fmt=".2f", cmap=cmap, center=0, linewidths=0.5)
    plt.title("Матрица корреляции")
    plt.show()

    return corr_df


def IQR_outliers_remove(df: pd.DataFrame, columns: list[str]) -> pd.DataFrame:
    for column in columns:
        Q1 = df[column].quantile(0.25)
        Q3 = df[column].quantile(0.75)
        IQR = Q3 - Q1
        df = df[(df[column] >= Q1 - 1.5 * IQR) & (df[column] <= Q3 + 1.5 * IQR)]
    return df


def plot_boxplots(df):
    numeric_columns = df.select_dtypes(include=['number']).columns

    if len(numeric_columns) == 0:
        print("Нет количественных столбцов для построения боксплотов.")
        return

    plt.figure(figsize=(len(numeric_columns) * 4, 6))
    df[numeric_columns].boxplot(rot=45)
    plt.title("Boxplots for Numeric Columns")
    plt.show()


def plot_hist(df):
    numeric_columns = df.select_dtypes(include=['number']).columns

    if len(numeric_columns) == 0:
        print("Нет количественных столбцов для построения гистограм.")
        return

    plt.figure(figsize=(len(numeric_columns) * 4, 6))
    df[numeric_columns].hist()
    plt.title("Hist for Numeric Columns")
    plt.show()


In [3]:
df = pd.read_excel("Датасет дано (1) (1).xlsx")
df.columns = df.columns.str.lower().str.replace(" ", "_")

In [4]:
# Создаем кпд

df['kpd'] = (df['useful_chats_fact'] + df['useful_calls_fact'] + df['avail_calls_fact'] + df['avail_chats_fact'] + df['avail_chat_call_fact'] + df["education_fact"] + df["additional_fact"]) / df["all_smena_plan"]

In [5]:
# Оставляем ток ТК + IQR чистка по кпд

df = df[df['emp_type'] == 'ТК']
df = IQR_outliers_remove(df[df['emp_type'] == 'ТК'], ['kpd'])

In [6]:
# Мат. Модель

now = df[df['grafik'].isin(['2/2', '5/2'])].dropna(subset=['kpd'])

now['2/2'] = (now['grafik'] == '2/2').astype(int)

kpd_group1 = now[now['2/2'] == 1]['kpd']
kpd_group2 = now[now['2/2'] == 0]['kpd']

t_stat, p_value = stats.mannwhitneyu(kpd_group1, kpd_group2, alternative='two-sided')

print(f"U-статистика: {t_stat:.4f}, p-значение: {p_value:.4f}")

U-статистика: 704905404.5000, p-значение: 0.0000


In [7]:
# Проверка на нормальность данных
print(norm(kpd_group1), norm(kpd_group2))

False False


In [8]:
print("Медианы:")
print('2: ', kpd_group2.median()*100)
print('1: ', kpd_group1.median()*100)

Медианы:
2:  85.58024691358025
1:  86.33101851851852


In [10]:
# Проверка на устойчивость по business_line

for i in df['business_line'].unique():
    now = df[df['business_line'] == i][df['grafik'].isin(['2/2', '5/2'])].dropna(subset=['kpd'])

    now['2/2'] = (now['grafik'] == '2/2').astype(int)

    kpd_group1 = now[now['2/2'] == 1][now['kpd'] < 1000]['kpd']
    kpd_group2 = now[now['2/2'] == 0][now['kpd'] < 1000]['kpd']
    if len(kpd_group1) != 0 and 0 != len(kpd_group2):
        t_stat, p_value = stats.mannwhitneyu(kpd_group1, kpd_group2, alternative='two-sided')
        print(f"{i}: p-значение: {p_value:.4f}, кол-во строк: {now.shape[0]}")

Mobile: p-значение: 0.0000, кол-во: 4710
Инвестиции: p-значение: 0.0000, кол-во: 3748
Депозитные продукты: p-значение: 0.0000, кол-во: 33266
SME: p-значение: 0.0000, кол-во: 10688
Кросс-функции: p-значение: 0.0000, кол-во: 3774
Кредитные продукты: p-значение: 0.0000, кол-во: 10242
Страховая: p-значение: 0.0000, кол-во: 1884
Premium Банк: p-значение: 0.0009, кол-во: 1939
Нефинансовые продукты: p-значение: 0.0000, кол-во: 1463


In [12]:
# Проверка на устойчивость по gender

for i in df['gender_desc'].unique():
    now = df[df['gender_desc'] == i][df['grafik'].isin(['2/2', '5/2'])].dropna(subset=['kpd'])

    now['2/2'] = (now['grafik'] == '2/2').astype(int)

    kpd_group1 = now[now['2/2'] == 1][now['kpd'] < 1000]['kpd']
    kpd_group2 = now[now['2/2'] == 0][now['kpd'] < 1000]['kpd']
    if len(kpd_group1) != 0 and 0 != len(kpd_group2):
        t_stat, p_value = stats.mannwhitneyu(kpd_group1, kpd_group2, alternative='two-sided')
        print(f"{i}: p-значение: {p_value:.4f}, кол-во строк: {now.shape[0]}")

Женщина: p-значение: 0.0000, кол-во строк: 45523
Мужчина: p-значение: 0.0000, кол-во строк: 26191
